# Churn Modelling Data Set Analysis Using ANN

## Data Preprocessing

In [1]:
# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [3]:
# Below Info Function will print detail information on the dataset that was imported in the previous step.
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### The above Info function tells us that there are 14 columns and 10,000 rows. This dataset contains 13 independent variables and one dependent variable (“Exited” column). Now let’s separate dataset into X(independent variables) and y(dependent variable). In X we’ll keep from third column to 13th (since we don’t need “RowNumber”,”CustomerId” and “Exited”) and in y we only need “Exited” field.

In [4]:
# Splitting the Data Set into Dependent and Independent Variable.
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

### We have two categorical variables (“Country” and “Gender” variable) in our data. We need to encode them. I used labelencoder and onehotencoder of scikit-learn package to achieve this. Note that I removed a dummy variable of “Country” to avoid falling into dummy variable trap.

In [5]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]


In [6]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


### Feature scaling is a method used to standardize the range of independent variables or features of data.

In [7]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


### Now our data is well preprocessed and now we will build the artificial neural network.

## Importing Tensorflow and Keras Library To Build ANN

### To start with building the ANN we will first import Tensorflow and Kreas Library.

In [8]:
# Part 2 - Now let's make the ANN!
import tensorflow as tf

### The sequential module is required to initialize the ann and dense module is required to add layers to it. 

In [9]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense


Using TensorFlow backend.


### Now we will initialize the deep learning model as a sequence of layers

In [10]:
# Initialising the ANN
classifier = Sequential()


### I have defined 6 units in the first hidden layer and used rectifying linear function(ReLu) as the activation function. And since input dimensions(no. of features) are 11 we will define input_dim=11.

In [11]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 11))


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
  from ipykernel import kernelapp as app


In [12]:
# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  from ipykernel import kernelapp as app


### Adding the output layer....It will have one output unit and we will use sigmoid function since it’s a binary classification.

In [13]:
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  from ipykernel import kernelapp as app


In [14]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [15]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Epoch 1/100
8000/8000 [==============================] - 1s 172us/step - loss: 0.4873 - acc: 0.7947 0s - loss: 0.5370 - 
Epoch 2/100
8000/8000 [==============================] - 1s 139us/step - loss: 0.4298 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 151us/step - loss: 0.4255 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 136us/step - loss: 0.4215 - acc: 0.8074
Epoch 5/100
8000/8000 [==============================] - 1s 145us/step - loss: 0.4188 - acc: 0.8210
Epoch 6/100
8000/8000 [==============================] - 1s 134us/step - loss: 0.4164 - acc: 0.8275
Epoch 7/100
8000/8000 [==============================] - 1s 138us/step - loss: 0.4145 - acc: 0.8294
Epoch 8/100
8000/8000 [==============================] - 1s 132us/step - loss: 0.4127 - acc: 0.8310
Epoch 9/100
8000/8000 [==============================] - 1s 132us/step - loss: 0.4114 - acc: 0.8330
Epoch 10/100
8000/8000 [==============================] - 1s 135us/step - loss:

Epoch 82/100
8000/8000 [==============================] - 1s 146us/step - loss: 0.4005 - acc: 0.8355
Epoch 83/100
8000/8000 [==============================] - 1s 141us/step - loss: 0.4002 - acc: 0.8361
Epoch 84/100
8000/8000 [==============================] - 1s 149us/step - loss: 0.4002 - acc: 0.8360
Epoch 85/100
8000/8000 [==============================] - 1s 140us/step - loss: 0.4005 - acc: 0.8359
Epoch 86/100
8000/8000 [==============================] - 1s 133us/step - loss: 0.4000 - acc: 0.8355
Epoch 87/100
8000/8000 [==============================] - 1s 133us/step - loss: 0.4004 - acc: 0.8364
Epoch 88/100
8000/8000 [==============================] - 1s 131us/step - loss: 0.4003 - acc: 0.8369
Epoch 89/100
8000/8000 [==============================] - 1s 131us/step - loss: 0.4004 - acc: 0.8352
Epoch 90/100
8000/8000 [==============================] - 1s 142us/step - loss: 0.4003 - acc: 0.8357
Epoch 91/100
8000/8000 [==============================] - 1s 137us/step - loss: 0.4002 - ac

In [16]:
# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)


In [17]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [18]:
cm

array([[1547,   48],
       [ 273,  132]])

### Merging Test Data and Predicted Test Results. Writing the combined data set into CSV File.

In [19]:
df1 = pd.DataFrame(X_test)
df2 = pd.DataFrame(y_pred)
submission = pd.concat([df1,df2],axis = 1)
submission.to_csv('submission.csv', index=False)
submission

,0,1,2,3,4,5,6,7,8,9,10,0
0,1.754865,-0.573694,-0.552043,-1.091687,-0.368904,1.044737,0.879303,-0.921591,0.642595,0.968738,1.610857,False
1,-0.569844,-0.573694,-1.314903,-1.091687,0.109617,-1.031415,0.429722,-0.921591,0.642595,-1.032270,0.495870,False
2,-0.569844,1.743090,0.571630,-1.091687,0.301026,1.044737,0.308583,-0.921591,0.642595,0.968738,-0.424787,False
3,-0.569844,-0.573694,1.416961,0.916013,-0.656016,-0.339364,0.575336,-0.921591,-1.556190,-1.032270,-0.187777,False
4,1.754865,-0.573694,0.571630,0.916013,-0.081791,0.006661,1.389611,0.809503,0.642595,0.968738,0.616842,False
5,-0.569844,1.743090,0.200509,-1.091687,1.736588,-0.685390,1.590021,0.809503,0.642595,-1.032270,-0.019302,True
6,-0.569844,1.743090,-0.624205,0.916013,-0.464608,-1.723466,-0.164023,0.809503,-1.556190,0.968738,1.045871,False
7,-0.569844,1.743090,-0.149995,-1.091687,-0.943129,0.352686,1.302385,0.809503,0.642595,0.968738,0.016166,False
8,-0.569844,-0.573694,-0.541734,-1.091687,2.406518,1.390762,-1.215717,-0.921591,-1.556190,0.968738,-1.511970,False
9,-0.569844,-0.573694,-2.005601,-1.091687,2.310814,-1.377440,1.426618,0.809503,-1.556190,-1.032270,0.705412,True
